# Experiment Runner: HLE Analysis with DeepSeek

このノートブックは設定ファイルに基づいて推論プロセス全体をオーケストレーションします

## 1. Setup and Configuration

In [ ]:
import os
import sys
from pathlib import Path

# --- 環境判別とセットアップ ---
# Colab環境かどうかを判定
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    print("✅ Colab環境を検出しました。セットアップを開始します。")
    from google.colab import drive, userdata
    drive.mount('/content/drive')

    # --- Google Driveのパスに合わせて変更してください ---
    PROJECT_ROOT = '/content/drive/MyDrive/llm-hle-experiment'
    # ---------------------------------------------------------

    os.chdir(PROJECT_ROOT)

    print("\n📦 必要なライブラリをインストールします...")
    !pip install -r requirements.txt -q | tail -n 1

    print("\n🔑 Hugging Face APIキーを設定します...")
    os.environ['HUGGINGFACE_API_KEY'] = userdata.get('HF_TOKEN')

else:
    print("✅ ローカル環境を検出しました。セットアップを開始します。")
    #  'notebooks' ディレクトリにいたら、プロジェクトルートに移動
    if Path.cwd().name == 'notebooks':
        os.chdir('..')

    PROJECT_ROOT = Path.cwd()

    print("\n🔑 .envファイルからHugging Face APIキーを読み込みます...")
    from dotenv import load_dotenv
    load_dotenv(Path(PROJECT_ROOT) / '.env')

# Pythonの検索パスに 'src' ディレクトリ追加
sys.path.append(str(Path(PROJECT_ROOT) / 'src'))

print(f"\n▶️ 作業ディレクトリ: {Path.cwd()}")
print("🎉 セットアップが完了しました。")

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from tqdm.auto import tqdm

from utils import load_config, load_prompt_template, ensure_dir
from data_loader import load_hle_dataset
from model_handler import ModelHandler

# APIキーの存在確認
hf_token_check = os.getenv('HUGGINGFACE_API_KEY')
print(f"Hugging Face Token Loaded: {'Yes' if hf_token_check else 'No'}")
if hf_token_check:
    print(f"Token starts with: {hf_token_check[:5]}...")

# --- 実験設定 ---
CONFIG_FILE = 'configs/deepseek_r1_0528_qwen3_8b_exp01_token32000_question_ids.yml' # @param 新しい実験用設定ファイル

print(f"\nLoading configuration from: {CONFIG_FILE}")
config = load_config(CONFIG_FILE)

# Promptsのロード
mc_prompt_template = load_prompt_template('mc_system_prompt')
em_prompt_template = load_prompt_template('em_system_prompt')

print("Configuration and prompts loaded successfully.")

## 2. Load Dataset

In [ ]:
dataset = load_hle_dataset(config['DATASET_NAME'])

# インデックス指定でフィルタリング
question_indices = config.get('QUESTION_INDICES', None)
if question_indices:
    # インデックスでサブセット作成
    subset_dataset = dataset.select(question_indices)
    print(f"Dataset filtered by indices: {question_indices}")
else:
    # 従来通りのサンプル数制限または全件
    num_samples = config.get('NUM_SAMPLES_TO_RUN', None)
    if num_samples is not None:
        subset_dataset = dataset.select(range(num_samples))
        print(f"Dataset selected by sample count: {num_samples}")
    else:
        subset_dataset = dataset
        print(f"Using full dataset")

print(f"Dataset loaded. Running on {len(subset_dataset)} samples.")
display(subset_dataset)

In [ ]:
# 冒頭2件のサンプルを確認
subset_dataset[:2]

## 3. Load Model and Tokenizer

In [ ]:
model_handler = ModelHandler(
    model_name=config['MODEL_NAME'],
    hf_token=os.getenv('HUGGINGFACE_API_KEY')
)

## 4. Run Inference Loop

In [ ]:
results = []

for example in tqdm(subset_dataset):
    if example['answer_type'] == 'multiple_choice': # Multiple Choice
        prompt_template = mc_prompt_template
    else: # Exact Match
        prompt_template = em_prompt_template
    final_prompt = prompt_template.format(question=example['question'])

    # モデルからの応答を生成
    raw_output = model_handler.generate(
        prompt=final_prompt,
        max_length=config.get('MAX_LENGTH', 4096),
        temperature=config.get('TEMPERATURE', 1.0),
        top_p=config.get('TOP_P', 1.0),
        top_k=config.get('TOP_K', 50)
    )

    # 応答を解析
    parsed_data = model_handler.parse_output(raw_output)

    # 結果を辞書形式で保存
    record = {
        "id": example['id'],
        "question": example['question'],
        "ground_truth": example['answer'],
        "model_answer": parsed_data.get('answer'),
        "confidence": parsed_data.get('confidence'),
        "explanation": parsed_data.get('explanation'),
        "think_process": parsed_data.get('think_process'),
        "parse_error": parsed_data.get('error'),
        "raw_output": raw_output
    }
    results.append(record)

## 5. Save Results

In [ ]:
results_df = pd.DataFrame(results)

# 出力ディレクトリとファイル名を設定
output_dir = config['output_path']
config_name = Path(CONFIG_FILE).stem # 拡張子なしのファイル名
output_csv_path = Path(output_dir) / f"{config_name}_results.csv"

# 出力ディレクトリを確認し、存在しない場合は作成する
ensure_dir(output_dir)

# 結果をCSVファイルに保存
results_df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

print(f"Results successfully saved to: {output_csv_path}")
display(results_df.head())